In [1]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from PIL import Image, ImageEnhance
#import ddddocr
import smtplib
from email.mime.text import MIMEText
import requests
import json
import faker

In [2]:
def imput_information():
    with open("IEEE.txt", "r") as file:
        content = file.read()
        content_list = content.split("\n")
    element1 = content_list[0]
    element2 = content_list[1]
    element3 = content_list[2]
    element4 = content_list[3]
    element5 = content_list[4]
    element6 = content_list[5]
    corpid = element1           #  企业id值
    corpsecret = element2       # 应用secret值
    touser = element3           # 多个用户用|隔开
    IEEE_ID = element4          # IEEE 账号
    IEEE_PW = element5          # IEEE 密码
    IEEE_WB = element6          # IEEE 网址
    toparty = "shaqima"         # 多个部门用|隔开
    totag = "SCI"               # 多个标签用|隔开
    agentid_ceshi = 1000005     # 应用id,agentid
    type_image = "image"        # （image）（voice）（video）(file)
    return corpid, corpsecret, touser, IEEE_ID, IEEE_PW, IEEE_WB, toparty, totag, agentid_ceshi

In [3]:
corpid, corpsecret, touser, IEEE_ID, IEEE_PW, IEEE_WB, toparty, totag, agentid_ceshi = imput_information()

In [4]:
get_act_url = "https://qyapi.weixin.qq.com/cgi-bin/gettoken?corpid={}&corpsecret={}".format(corpid,corpsecret)
act_res = requests.get(url=get_act_url).json()
access_token = act_res["access_token"]

fk = faker.Faker()
url = f"https://qyapi.weixin.qq.com/cgi-bin/message/send?access_token={access_token}&random=69152"

payload = json.dumps({
   "touser": touser,
   "toparty": toparty,
   #"totag": totag,
   "msgtype": "text",
   "agentid": agentid_ceshi,
   "text": {"content": "已备份成功2，请查收！"},
   "safe": 0,
   "enable_id_trans": 0,
   "enable_duplicate_check": 0})
headers = {
   'User-Agent': fk.user_agent(),
   'Content-Type': 'application/json'}

response = requests.request("POST", get_act_url, headers=headers, data=payload)


def get_access_token():
    get_act_url = "https://qyapi.weixin.qq.com/cgi-bin/gettoken?corpid={}&corpsecret={}".format(corpid,corpsecret)
    act_res = requests.get(url=get_act_url).json()
    access_token = act_res["access_token"]
    return access_token

def send_text_message(message):
    send_text_url = "https://qyapi.weixin.qq.com/cgi-bin/message/send?access_token={}".format(get_access_token())
    data={
        "touser" : touser, #接收消息的用户
        "msgtype" : "text", #消息类型
        "agentid" : agentid_ceshi, #应用id
        "text" : {
            "content" : message #消息内容
            },
        "safe":0, #0为公开信息，1为保密信息
        }
    text_message_res = requests.post(url=send_text_url,data=json.dumps(data)).json()
    return text_message_res

In [5]:
def send_myemail(Emailtext):
    mail_host = 'smtp.qq.com'
    mail_user = '1219310749'
    mail_pass = 'oxcqnxkxtxxsifdd'
    sender = '1219310749@qq.com'
    # 邮件接受方邮箱地址，注意需要[]包裹，这意味着你可以写多个邮件地址群发
    receivers = ['zhangnieng@126.com']

    message = MIMEText(Emailtext, 'plain', 'utf-8')
    message['Subject'] = 'Manuscript Review Status'
    message['From'] = sender
    message['To'] = receivers[0]

    try:
        smtpObj = smtplib.SMTP()
        # 连接到服务器
        smtpObj.connect(mail_host, 25)
        # 登录到服务器
        smtpObj.login(mail_user, mail_pass)
        # 发送
        smtpObj.sendmail(sender, receivers, message.as_string())
        # 退出
        smtpObj.quit()
        print('success')
    except smtplib.SMTPException as e:
        print('error', e)

In [6]:
# send_myemail("Emailtext")
# send_text_message("message")

In [6]:
#  chrome自动化  chromedriver的位置
SS = Service("./chromedriver.exe")

browser = webdriver.Chrome(service=SS)

# browser.maximize_window()

#  打开 投稿系统  ieee-tii  根据自己的改正
browser.get(IEEE_WB)

# ==================↓不确定因素↓================== #
time.sleep(10)
#  点击 接受按钮（有时网页会弹出一个  接受按钮）
browser.find_element(By.XPATH, '//*[@id="onetrust-close-btn-container"]/button').click()
# ==================↑不确定因素↑================== #

#  输入 账号，密码，点击登陆  自行输入IEEE投稿 账号和密码
browser.find_element(By.NAME, "USERID").send_keys(IEEE_ID)
browser.find_element(By.NAME, "PASSWORD").send_keys(IEEE_PW)

#  点击 登录按钮
browser.find_element(By.ID, 'logInButton').click()

#  点击 Author 按钮
browser.find_element(By.XPATH, '//*[@id="header"]/div/div[3]/div/ul/li[2]/a').click()

In [9]:
last_elements = ['Draft', 'Awaiting AE Assignment', 'Assign additional Reviewers', 'Awaiting Review Scores', 'AA']

In [ ]:
# for j in range(3):  #  循环两次
while True:         # 一直循环
    # 获取 状态个数
    elements = browser.find_elements(By.CLASS_NAME, "pagecontents")
    my_list = [element.text for element in elements]
    my_listX = []
    for i in my_list:
        if i:
            my_listX.append(i)
    my_list = my_listX
    if "a revision has been submitted" in my_list:
        my_list = [item for item in my_list if item!= 'a revision has been submitted']
  
    if len(my_list)!= len(last_elements):
        my_position = [i for i, x in enumerate(my_list) if x not in last_elements]
        for m in my_position:
            Title = browser.find_element(By.XPATH, f'//*[@id="queue_{m}"]/td[3]').text
            Title = Title.replace("View Submission"," ")
            print(f"New Submit \n {Title}")
        
            Emailtext = f"New Submit \n {Title}"
            send_myemail(Emailtext)
            send_text_message(Emailtext)
        last_elements = my_list[:] # 备份当前元素
  
    else:
        # 检测 状态是否改变 
        for i, element in enumerate(my_list):
            if element != last_elements[i]:
                Title = browser.find_element(By.XPATH, f'//*[@id="queue_{i}"]/td[3]').text
                Title = Title.replace("View Submission"," ")
                print(f"{Title} \n Last: {last_elements[i]} \n Now: {element}")

                Emailtext = f"{Title} \n Last: {last_elements[i]} \n Now: {element}"
                send_text_message(Emailtext)
                send_myemail(Emailtext)
        last_elements = my_list[:]
    
    time.sleep(300)
    browser.refresh()

In [ ]:

for j in range(3):  #  循环两次
# while True:         # 一直循环
    # 获取 状态个数
    #elements = browser.find_elements(By.CLASS_NAME, "pagecontents")
    #my_list = [element.text for element in elements]
    my_list = ['','a','b','c','d']
    my_listX = []
    for i in my_list:
        if i:
            my_listX.append(i)
    my_list = my_listX
    if "a" in my_list:
        my_list = [item for item in my_list if item!= 'a']
  
    if len(my_list)!= len(last_elements):
        my_position = [i for i, x in enumerate(my_list) if x not in last_elements]
        for m in my_position:
            Title = browser.find_element(By.XPATH, f'//*[@id="queue_{m}"]/td[3]').text
            Title = Title.replace("View Submission"," ")
            print(f"New Submit \n {Title}")
        
            Emailtext = f"New Submit \n {Title}"
            send_myemail(Emailtext)
            send_text_message(Emailtext)
        last_elements = my_list[:] # 备份当前元素
  
    else:
        # 检测 状态是否改变 
        for i, element in enumerate(my_list):
            if element != last_elements[i]:
                Title = browser.find_element(By.XPATH, f'//*[@id="queue_{i}"]/td[3]').text
                Title = Title.replace("View Submission"," ")
                print(f"{Title} \n Last: {last_elements[i]} \n Now: {element}")

                Emailtext = f"{Title} \n Last: {last_elements[i]} \n Now: {element}"
                send_text_message(Emailtext)
                send_myemail(Emailtext)
        last_elements = my_list[:]
    
    time.sleep(300)
    browser.refresh()

In [17]:
my_list = ['a','b','b','c','','d']
my_listX = []
for i in my_list:
    if i:
        my_listX.append(i)
my_list = my_listX
if "a" in my_list:
    my_list = [item for item in my_list if item!= 'a']
print(my_list)

['b', 'b', 'c', 'd']


In [18]:
last_elements = ['b','c','d']
if len(my_list)!= len(last_elements):
    my_position = [i for i, x in enumerate(my_list) if x not in last_elements]
    for m in my_position:
        print(f"New Submit \n {m}")
    last_elements = my_list[:]
print(my_list)
print(my_position)

['b', 'b', 'c', 'd']
[]
